In [1]:
%load_ext lab_black
%cd ../..

/mnt/h/hev/log-analytics


In [2]:
import numpy as np
import pandas as pd

In [3]:
csvs = [
    pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05_1-ver4.csv"),
    pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-out_ver2.csv"),
    pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver2.csv"),
    pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05_4-ver4.csv"),
    # pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_5-out_ver1.csv"),
    # pd.read_csv("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4-out_ver1.csv"),
]

In [4]:
levels = np.stack([csv.level.to_numpy() for csv in csvs])

In [5]:
N = len(csvs[0])

In [6]:
out_levels = []
for i in range(N):
    kt = False
    ll = [0 for _ in range(8)]
    for j in range(len(csvs)):
        if levels[j, i] in [2, 4, 6]:
            out_levels.append(levels[j, i])
            kt = True
            break
        else:
            ll[levels[j, i]] += 1

    if not kt:
        if max(ll) == 1:
            out_levels.append(levels[0, i])
        elif ll[7] >= 2:
            out_levels.append(7)
        else:
            out_levels.append(max(range(8), key=lambda i: ll[i]))

In [7]:
df = pd.DataFrame({"id": csvs[0].id.to_list(), "level": out_levels})

In [8]:
df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [25]:
df.to_csv("results/ens-ver5.csv", index=False)

|version|note|score|
|---|---|---|
|1|7.3.3 + 7.4.4 + 8.4.1|0.9199602809|
|2|7.3.2 + 7.4.4 + 8.4.1|0.9208938205|
|3|7.1~7.5 + 8.4.1|0.9178614787|

In [9]:
tdf = pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver2.csv")

In [10]:
tlevels = tdf.level.to_numpy()

In [11]:
olevels = df.level.to_numpy()

In [12]:
(olevels == 7).nonzero()[0][:100]

array([  1766,   3780,   6003,   7890,  11515,  12639,  15160,  16932,
        19574,  22246,  26611,  27854,  30852,  37174,  38180,  38592,
        41200,  45028,  45581,  50471,  50899,  51856,  51887,  51953,
        53736,  54247,  56488,  58126,  58797,  61121,  63975,  64630,
        65387,  65980,  66248,  66649,  68203,  68300,  68433,  69985,
        70089,  70526,  74637,  76838,  78292,  79586,  79909,  81139,
        82047,  82287,  84329,  85474,  89806,  93830,  93934,  97456,
        98325,  99553, 100227, 100362, 100875, 101715, 102363, 103617,
       104784, 109667, 110436, 110962, 112975, 113459, 117120, 117166,
       117614, 119211, 119436, 119759, 122720, 124131, 127593, 132314,
       133353, 133911, 134165, 137872, 138723, 142588, 143352, 144966,
       146134, 146998, 147281, 148482, 148979, 152604, 153382, 153891,
       156521, 156913, 158447, 159643])

In [13]:
(tlevels == 7).nonzero()[0][:100]

array([  6003,   7890,   9928,  11151,  11515,  15160,  16932,  19574,
        22246,  26611,  27854,  30852,  37174,  38180,  38592,  41200,
        50471,  50899,  51887,  51953,  53736,  54247,  56488,  58126,
        58797,  61121,  64630,  65387,  66248,  66649,  68203,  68300,
        69985,  70089,  74637,  76838,  79586,  79909,  81139,  82047,
        82287,  84329,  85474,  88037,  89806,  93830,  93934,  97456,
        98325, 100362, 100875, 101297, 102363, 103617, 113459, 117120,
       117614, 119211, 119436, 119759, 122720, 124131, 133353, 133911,
       134165, 137872, 138723, 143352, 144966, 146134, 146998, 147281,
       148482, 148979, 152604, 153382, 153891, 158447, 159643, 160810,
       165543, 169905, 170457, 175138, 182070, 182445, 184448, 190307,
       193314, 194715, 198598, 201568, 202879, 203311, 203669, 208260,
       209443, 210211, 210933, 213041])

In [14]:
sdf = pd.read_csv("data/ori/test.csv")

In [24]:
sdf.full_log[45581]

'level : 3, log : received received-value bytes, expected expected-value bytes'

In [16]:
trdf = pd.read_csv("data/ori/train.csv")

In [17]:
def find_all(text):
    cnt = 0
    for i in range(len(trdf)):
        if text in trdf.full_log[i]:
            print(f"{i:8} | {trdf.level[i]} | {trdf.full_log[i]}")
            print()
            cnt += 1
        if cnt > 5:
            break

In [18]:
find_all("ANOM_ABEND")